In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)  

from azure.datalake.store import core, lib, multithread
import os
import pathlib

import argparse
import warnings  
import time
from scipy import stats
from datetime import datetime as dt
import collections
import pickle
import tempfile

import pickle
import os
import azureml.core
from azureml.core.compute import ComputeTarget, DataFactoryCompute
from azureml.exceptions import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.pipeline.core import Pipeline
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.core import Dataset

from azureml.pipeline.steps import DataTransferStep

from msrest.exceptions import HttpOperationError


from azureml.pipeline.core import Pipeline, PipelineRun, PipelineData, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.core import Workspace, Experiment, Datastore

from azureml.widgets import RunDetails


In [3]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(tenant_id="...", # tenantID
                                    service_principal_id="...", # clientId
                                    service_principal_password="...") # clientSecret


In [4]:
from azureml.core import Workspace

subscription_id = '...'
resource_group = '...'
workspace_name = '...'


ws = Workspace(subscription_id, resource_group, workspace_name, auth = sp)

In [5]:
blob_datastore_name='rbru' # Name of the datastore to workspace

container_name=os.getenv("BLOB_CONTAINER", "rbru") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "qrbexpdlg2weu01") # Storage account name

# Storage account access key
account_key=os.getenv("BLOB_ACCOUNT_KEY", "...") 

if blob_datastore_name not in ws.datastores.keys():
    blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)
    print("Registered datastore with name: %s" % blob_datastore_name)
else:
    print("Datastore {} already exists".format(blob_datastore_name))

Datastore rbru already exists


In [6]:
ws.datastores.keys()

dict_keys(['workspaceartifactstore', 'eudata', 'rbru', 'rbrumassdl', 'rbrucisdl', 'rb_health_russia', 'azureml_globaldatasets', 'workspaceblobstore', 'workspacefilestore'])

In [7]:
blob_datastore_source = ws.datastores['rb_health_russia']
blob_datastore = ws.datastores['rbru']

## Pipleline

In [2]:
import os

# Create a folder 
script_folder = 'mass_explode'
os.makedirs(script_folder, exist_ok=True)

print(script_folder)

mass_explode


In [9]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

env = Environment("pipeline-env")
env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
env.docker.enabled = True # Use a docker container

dependencies = CondaDependencies.create(conda_packages=['scikit-learn','pandas'],
                                             pip_packages=['azureml-defaults', 'xlrd', 'geopy',
                                                           'googlemaps', 'haversine', 'shapely', 
                                                           'azureml-dataprep[pandas]', 'openpyxl',
                                                            'ortools==6.7.4973'])
env.python.conda_dependencies = dependencies

env.register(workspace=ws)
registered_env = Environment.get(ws, 'pipeline-env')
pipeline_run_config = RunConfiguration()

pipeline_run_config.target =  'fff555'  #aaa111  
pipeline_run_config.environment = registered_env

### _final poi_step_

In [31]:
%%writefile $script_folder/poi_volga.py
from azureml.core import Run
import pandas as pd
import numpy as np
import os, argparse, time, warnings, re 
import urllib
import requests
import geopy.distance
run = Run.get_context()

def data_prep(store_data, poi_data):
    
    poi_data.fillna(poi_data[[i for i in poi_data.columns if  i.islower()]  + ['Rating']].fillna(0), inplace=True) #fillna cols with google types

    store_data['Address'] = np.where((store_data['FiasAccuracy'] =='Корректный') & (store_data['FiasAddress'].notnull()), 
                             store_data['FiasAddress'], store_data['SourceAddress'])

    regions = ['Moscow', 'Center', 'Ural', 'Far East', 'Volga', 'South', 'North-West', 'Siberia']
#     store_data = store_data[store_data['Macroregion'].isin(['Volga'])]

    store_data=store_data.loc[(store_data['Active (Y/N)']==True) & (store_data['Channel'].isin(['LKA','NKA']))]
    print(store_data.shape)

    store_data = store_data[pd.notnull(store_data['Store ID'])]
    store_data['Store ID'] =store_data['Store ID'].astype(int)
    store_data['Store ID'] =store_data['Store ID'].apply(lambda x:str(x)+"_")

    store_data_final=store_data.drop_duplicates(['Store ID','City'])
    store_data_final=store_data_final.loc[store_data_final["Format"].isin(
                ['Super', 'Drug', 'Discounter', 'Kiosk', 'Hyper', 'Other', 'Alco Minimarket', 
                 'WS', 'Convenience Store', 'Ecomm', 'Pavilion','Petrol Station', 'Unknown']),:]

    store_data  = store_data[['Store ID','Address', 'Lat/Long', 'Channel', 'FiasAccuracy', 'Macroregion']] 
    store_data[['Широта','Долгота']]=store_data['Lat/Long'].str.split('/',expand=True).apply(lambda x: x.str.strip().astype('float'))
    store_data['Coordinates'] =  list(zip(store_data['Широта'], store_data['Долгота']))

    store_data.drop('Lat/Long', inplace =True, axis=1)
    store_data = store_data.dropna(subset=['Долгота', 'Широта']) #remove where lat, long = NaN
    print(store_data.shape)
    l = list(set(store_data['Store ID']) - set(poi_data['Store ID']))
    print(len(l))
    l =l[:200]         #  batch of 200 stores per run
    
    store_data = store_data[store_data['Store ID'].isin(l)]
    return store_data


#add Points of interest in radius 300 metres, from all pages 
def add_poi(df):
    main_api = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

    key ='...' #password
    radius = 300
    language ='ru'

    data =[]
    for i in  range(0, df.shape[0]): 
 
        print('Iteration POI:', i,  df['Store ID'].iloc[i])
        coords = df['Coordinates'].iloc[i]
        url = main_api + urllib.parse.urlencode({'location': re.sub(r'\s*, \s*', ',', str(coords)[1:-1]),
                                                 'key': key, 'radius': radius, 'language': language})
        time.sleep(2)
        data_json = requests.get(url).json()
        json_status = data_json['status']
        
        if json_status =='OK':
            poi=[]
            names =[]

            for k in data_json['results']:
                names.append(k['name'])

                for j in k['types']:
                    poi.append(j)
                    
            data.append(pd.merge(df.reset_index(drop= True),
                                 pd.DataFrame({k: poi.count(k) for k in poi}, index = [i]), 
                                 left_index=True, right_index=True))
             
            try:
                next_page_token = data_json["next_page_token"]
                
                while data_json["next_page_token"]:

                    url_nextpage = main_api + urllib.parse.urlencode({'key': key, 
                                                            'pagetoken': next_page_token})
                    #print(url_nextpage)
                    time.sleep(2)
                    data_json = requests.get(url_nextpage).json()

                    if data_json['status'] =='OK':
                        
                        for k in data_json['results']:
                            names.append(k['name'])

                            for j in k['types']:
                                poi.append(j)
                    #print('poi on next page:', {k: poi.count(k) for k in poi})

                    try:
                        next_page_token = data_json["next_page_token"]
                    except KeyError:
                        break
                        
                    continue
                    
            except KeyError:
                continue
                
        data.append(pd.merge(df.reset_index(drop= True),
                       pd.DataFrame({k: poi.count(k) for k in poi}, index = [i]), 
                       left_index=True, right_index=True))


    d =pd.concat(data, ignore_index=True)    

    cols = [i for i in d.columns if not i.islower()] + [i for i in d.columns if i.islower()]
    d = d[cols]
    d.drop_duplicates(['Store ID'], keep='last',inplace=True)
    d =d.reset_index(drop= True)
    d.fillna(d[[i for i in d.columns if  i.islower()]].fillna(0), inplace=True) #fillna cols with google types
    return d



#add features: 'Opening_hours', 'Rating', 'User_ratings_total', 'Price_level'

def add_features(df):
    main_api = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'

    key ='...'
    language ='ru'

    data=[]
    lat, lng, price_level, rating, user_ratings_total, opening_hours =([] for i in range(6))

    for i in  range(0, df.shape[0]):  
        #print('Query: ',i)
        input = df['Address'].iloc[i]
        url = main_api + urllib.parse.urlencode({'input':input, 'inputtype': 'textquery',
                                                 'key': key,'language': language,
                                                 'fields': 'place_id,photos,formatted_address,name,rating,price_level,opening_hours,user_ratings_total,geometry'})

        time.sleep(2)
        data_json = requests.get(url).json()
        # print(data_json)

        if data_json['status'] =='OK':

            try:
                place_id = data_json["candidates"][0]['place_id']
                #print(data_json['candidates'][0]["geometry"]['location']['lat'])


                #details
                details_api ='https://maps.googleapis.com/maps/api/place/details/json?'
                url_details = details_api + urllib.parse.urlencode({'place_id':place_id, 
                                                                     'key': key,'language': language})

                time.sleep(2)
                details_json = requests.get(url_details).json()
                #print('Details:', url_details)
                #df.loc[df.index[i], 'Lat'] = data_json['candidates'][0]["geometry"]['location']['lat']
                #df.loc[df.index[i], 'Lng'] = data_json['candidates'][0]["geometry"]['location']['lng']

                df.loc[df.index[i], 'Opening_hours'] = abs(int(details_json['result']["opening_hours"]["periods"][1]['close']['time'])/100 -
                                     int(details_json['result']["opening_hours"]["periods"][1]['open']['time'])/100)



                #rating, price level
                df.loc[df.index[i], 'Rating'] =  data_json['candidates'][0]["rating"]
                df.loc[df.index[i], 'User_ratings_total'] = data_json['candidates'][0]["user_ratings_total"]
                df.loc[df.index[i], 'Price_level'] =  data_json['candidates'][0]["price_level"]


            except KeyError:
                continue
            except IndexError:
                continue
    
    return df


#check if there is a road next to store

def add_roads(df):
    main_api = 'https://roads.googleapis.com/v1/nearestRoads?'

    key ='...'
   
    road_points=[]

    df['Долгота'] = df['Долгота'].astype('float')
    df['Широта']= df['Широта'].astype('float')
    df['Coordinates'] =  list(zip(df['Широта'], df['Долгота']))

    for i in  range(0, df.shape[0]):
        #print('Query: ',i)
        coords = df['Coordinates'].iloc[i]
        url = main_api + urllib.parse.urlencode({'points':re.sub(r'\s*, \s*', ',', str(coords)[1:-1]),
                                                 'key': key})

        time.sleep(2)
        data_json = requests.get(url).json()
        if data_json:
            try:
                for j in data_json["snappedPoints"]:
                    road_points.append((j['location']['latitude'], j['location']['longitude']))

                min_dist =  geopy.distance.geodesic(road_points[0], coords).meters
                for k in road_points:
                    if min_dist> geopy.distance.geodesic(k, coords).meters:
                        min_dist = geopy.distance.geodesic(k, coords).meters
                df.loc[df.index[i], 'Distance to the nearest road'] = min_dist
            except KeyError:
                continue

    return df


def main(input_path, input_folder):
    
    #read data
    
    poi_data  = pd.read_csv(os.path.join(input_path, input_folder, 'poi.csv'),
                                sep =',')
    for i in poi_data.columns:
        if re.match(r'^Unnamed', i) or re.match(r'^Column', i):
            poi_data.drop(i, axis=1, inplace =True)
    run.log('initial poi shape:', poi_data.shape)

   
    store_data = pd.read_csv(os.path.join(input_path, input_folder,'store_char.csv'),  
                                sep ='|',
                                usecols=["Store ID","Active (Y/N)","StoreName","City","Region",
                                'FiasAddress','SourceAddress', 'FiasAccuracy', 'Macroregion',
                                'Channel', 'Format',"Lat/Long","# of Check-outs","Coverage (Y/N)", 
                                "Brick/Sales Territory","# Check-outs for SWB","Chain Name",
                                "Parent Chain Name","Degree of Freedom"])
    
    #functions
    prep_data = data_prep(store_data, poi_data)
    final_df = prep_data.pipe(add_poi).pipe(add_features).pipe(add_roads)

    if 'Opening_hours' in final_df.columns:
        final_df['24h'] = [1 if i == 24 else 0 for i in final_df['Opening_hours']]
        
    if 'Distance to the nearest road' in final_df.columns:
        final_df['Nearest_road'] = [1 if i <= 30 else 0 for i in final_df['Distance to the nearest road']]
    
    final_df.fillna(final_df[[i for i in final_df.columns if  i.islower()]].fillna(0), inplace=True) #fillna cols with google types

    if not final_df.empty:
        poi_data=pd.concat([poi_data,final_df],ignore_index=True)
        poi_data.fillna(poi_data[[i for i in poi_data.columns if  i.islower()] + ['Rating', 'Nearest_road']].fillna(0), inplace=True) #fillna cols with google types

    else:
        pass
    '''
    final_df['Точность (ФИАС)'].fillna(store_data['FiasAccuracy'], inplace = True)
    final_df['Тип сети'].fillna(store_data['Channel'], inplace = True)
    
    final_df['Тип сети'] = final_df['Тип сети'].map({'Локальная сеть': 'LKA',  'Национальная сеть': 'NKA'})
    final_df.rename(columns ={'Тип сети':'Channel', 'Точность (ФИАС)': 'FiasAccuracy'}, inplace =True)
    final_df.drop(['Channel', 'FiasAccuracy', 'Distance to the nearest road'], inplace =True, axis=1)
    '''
    run.log('final_df shape:', poi_data.shape)

    
    #save output
    poi_data.to_csv(os.path.join(args.input_path, args.input_folder,'poi.csv'),
                    encoding='utf-8-sig')
   
    #----------------------------------------------------------------------------------
    
    
if __name__ == "__main__":
    
    parser = argparse.ArgumentParser('poi step')
    parser.add_argument("--input_path", type=str, help="input_path") #data_exp/
    parser.add_argument("--input_folder", type=str, help="input_folder") #raw_data
    #parser.add_argument("--master_data_folder", type=str, help="master_data_folder") #master_data

    args = parser.parse_args()

    run.log('Output directory:', os.path.join(args.input_path, args.input_folder))
    
    
    main(args.input_path, args.input_folder)

Overwriting mass_explode/poi_volga.py


In [3]:
%%writefile $script_folder/photos.py

#grab photos of store by its address using Places API, save photos to folder in Data Lake

from azureml.core import Run
import pandas as pd
import numpy as np
import os, argparse, re, time
import urllib
import requests, pathlib
from PIL import Image
import matplotlib.pyplot as plt
import joblib as jlb # parallelizing


parser = argparse.ArgumentParser('photo')
parser.add_argument("--input_photo", type=str, dest="input_photo")
parser.add_argument("--ncpus", type=int, default=5, help="number of cpus to use")

args = parser.parse_args()

path = pathlib.Path(args.input_photo)
ds_path = os.path.join(*path.parts[:-3]) 

run = Run.get_context()

df =  pd.read_csv(os.path.join(args.input_photo, 'preped_data.csv'))

def add_photo(index, folder):
    main_api = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
    key ='...'
    language ='ru'
    
    input = df['Address'].iloc[index]
    url = main_api + urllib.parse.urlencode({'input':input, 'inputtype': 'textquery',
                                             'key': key,'language': language,
                                             'fields': 'place_id,photos,geometry'})
    time.sleep(2)
    data_json = requests.get(url).json()
    if data_json['status'] =='OK':

        try:
            place_id = data_json["candidates"][0]['place_id']

            photo_id = data_json['candidates'][0]["photos"][0]['photo_reference']
            url_photo = 'https://maps.googleapis.com/maps/api/place/photo?' + urllib.parse.urlencode({'key': key, 
                                                                                                      'photoreference': photo_id,
                                                                                                      'maxwidth':400})

            time.sleep(2)
            print(url_photo)
            img = Image.open(requests.get(url_photo, stream=True).raw) 

            imgResult = img.resize((256, 256), resample = Image.BILINEAR).convert('RGB')
            imgResult.save(folder+'/'+ 'file_' + str(index)+ '.jpg')


        except KeyError:
            pass


jlb.Parallel(n_jobs=args.ncpus)(jlb.delayed(add_photo)(i, 
                                                       folder= ds_path+ '/Output/MASS/photo') 
                                for i in  range(0, df.shape[0]))



Writing mass_explode/photos.py


In [ ]:
# code for a single pipeline run:

blob_datastore = Datastore.get(ws, 'rbru') #not ws.datastores['rbru']!

data_path = DataPath(datastore= blob_datastore, path_on_datastore='data_exp/')
datapath_input = (PipelineParameter(name="input_datapath", default_value=data_path),
                  DataPathComputeBinding(mode='mount'))

                  #mode='download', path_on_compute=script_folder, overwrite=False)
    
param_1 = PipelineParameter(name="input_folder_param", default_value='raw_data/') 

poi_step = PythonScriptStep(
    name='poi_center',
    source_directory= script_folder, 
    script_name="poi_center.py",
    compute_target=pipeline_run_config.target, 
    inputs=[datapath_input],
    arguments=["--input_path", datapath_input, 
              "--input_folder", param_1],    
    runconfig = pipeline_run_config)

print("poi_step created")

pipeline = Pipeline(workspace=ws, steps=[poi_step])

pipeline.validate()

# Create an experiment and run the pipeline
pipeline_run = Experiment(workspace = ws, 
                          name = 'poi_center').submit(pipeline, regenerate_outputs=True)

#RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

In [393]:
for step_run in pipeline_run.get_children():
    print("{}: {}".format(step_run.name, step_run.get_metrics())) #14285;  1139 left for moscow

poi: {'Output directory:': '/mnt/batch/tasks/shared/LS_root/jobs/eu-ru-ds-ml-001/azureml/a09a0166-e98e-4a3d-b73e-2cae264b50ea/mounts/rbru/data_exp/raw_data/', 'initial poi shape:': '(14285, 128)', 'final_df shape:': '(14485, 128)'}


## _schedule pipeline_


In [32]:
blob_datastore = Datastore.get(ws, 'rbru') 

data_path = DataPath(datastore= blob_datastore, path_on_datastore='data_exp/')
datapath_input = (PipelineParameter(name="input_datapath", default_value=data_path),
                  DataPathComputeBinding(mode='mount'))

    
param_1 = PipelineParameter(name="input_folder_param", default_value='raw_data/') 

poi_step = PythonScriptStep(
    name='poi_volga',
    source_directory= script_folder, 
    script_name="poi_volga.py",
    compute_target=pipeline_run_config.target, 
    inputs=[datapath_input],
    arguments=["--input_path", datapath_input, 
              "--input_folder", param_1],    
    runconfig = pipeline_run_config)

print("poi_step created")
pipeline = Pipeline(workspace=ws, steps=[poi_step])

published_pipeline = pipeline.publish(name="POI_Pipeline_volga", version="1.0",
                                      description="adding 200 points every 25 minutes, region: Volga")

published_pipeline

poi_step created
Created step poi_volga [5bb93a9c][ddda619c-0f63-494a-94e0-fbd04667c4b2], (This step will run and generate new outputs)
Created data reference rbru_5441d7a5 for StepId [0e8f7995][cfd1f3bc-f130-4641-9a76-1dcdf8da730b], (Consumers of this data will generate new runs.)


Name,Id,Status,Endpoint
POI_Pipeline_volga,d39421dd-e759-4690-bb2b-a824d0e731d3,Active,REST Endpoint


In [33]:
from azureml.pipeline.core import Schedule, ScheduleRecurrence

recurrence = ScheduleRecurrence(frequency="Minute", interval=25)
schedule = Schedule.create(ws, name="POI_schedule_volga", pipeline_id=published_pipeline.id,
                          experiment_name="POI_schedule_volga", recurrence=recurrence)

In [37]:
pipelineruns = PipelineRun.get_pipeline_runs(ws, published_pipeline.id)
print(pipelineruns)
for t in pipelineruns:
    details = t.get_details()
    print(details['startTimeUtc'])
    #print(details['endTimeUtc'])
   # print(details['properties']['azureml.parameters'])

[Run(Experiment: POI_schedule_volga,
Id: 06c5886c-c1fc-49bb-82d0-d68f52c5cf1d,
Type: azureml.PipelineRun,
Status: Running)]
2021-03-23T11:56:51.444037Z


In [40]:
for i in schedule.get_pipeline_runs(): 
    for step_run in i.get_children():
        print("{}: {}: {}".format(i.id, step_run.name, step_run.get_metrics()))

7da3b420-6899-4583-a0d6-2d8b0b5dc264: poi_volga: {'Output directory:': '/mnt/batch/tasks/shared/LS_root/jobs/eu-ru-ds-ml-001/azureml/0930cb33-fea7-49a8-82ba-30f14f656a0a/mounts/rbru/data_exp/raw_data/', 'initial poi shape:': '(62220, 130)'}
61ffffe9-2da8-458c-805d-38b52798d0b5: poi_volga: {'Output directory:': '/mnt/batch/tasks/shared/LS_root/jobs/eu-ru-ds-ml-001/azureml/a09287a7-5a18-4ced-ae16-1e35700d54ba/mounts/rbru/data_exp/raw_data/', 'initial poi shape:': '(62020, 130)', 'final_df shape:': '(62220, 130)'}
326bc24f-8319-4536-b677-8d57b8196b6d: poi_volga: {'Output directory:': '/mnt/batch/tasks/shared/LS_root/jobs/eu-ru-ds-ml-001/azureml/22bbde6a-8ae4-4ce4-a98d-5dfed3eabab0/mounts/rbru/data_exp/raw_data/', 'initial poi shape:': '(61820, 130)', 'final_df shape:': '(62020, 130)'}
e14e04bb-8d23-4310-bc53-06b3c84fd752: poi_volga: {'Output directory:': '/mnt/batch/tasks/shared/LS_root/jobs/eu-ru-ds-ml-001/azureml/51e544e7-045d-4334-975d-26523b375b71/mounts/rbru/data_exp/raw_data/', 'ini

In [42]:
ss = Schedule.list(ws)
ss

[Pipeline(Name: POI_schedule_south,
 Id: 94ac63e3-d9f7-48af-a266-a5394fa95a32,
 Status: Active,
 Pipeline Id: 5427d66b-7557-4c66-a886-0445d301cf04,
 Recurrence Details: Runs every 25 Minutes)]

## _enable/ disable schedule_

In [41]:
#all schedules for this pipeline
schedules = Schedule.list(ws, pipeline_id= published_pipeline.id)
print("Found these schedules for the pipeline id {}:".format(published_pipeline.id))


for schedule in schedules: 
    print(schedule.id)
    if schedule.recurrence is not None:
        schedule_id = schedule.id
print("Schedule id: {}".format(schedule_id))
 
  #get schedule by id
fetched_schedule = Schedule.get(ws, schedule_id)
print("Using schedule with id: {}".format(fetched_schedule.id))


#disable this schedule (by id)

fetched_schedule.disable(wait_for_provisioning=True)
fetched_schedule = Schedule.get(ws, schedule_id)
print("Disabled schedule {}. New status is: {}".format(fetched_schedule.id, fetched_schedule.status))


Found these schedules for the pipeline id d39421dd-e759-4690-bb2b-a824d0e731d3:
1419cda1-8535-44f1-98ed-e5495510af43
Schedule id: 1419cda1-8535-44f1-98ed-e5495510af43
Using schedule with id: 1419cda1-8535-44f1-98ed-e5495510af43
Provisioning status: Completed
Disabled schedule 1419cda1-8535-44f1-98ed-e5495510af43. New status is: Disabled


In [560]:
#change reccurence

recurrence = ScheduleRecurrence(frequency="Minute", interval=45) # Runs every 45 minutes

fetched_schedule = Schedule.get(ws, schedule_id)

fetched_schedule.update(name="POI_schedule_north_west", 
                        description="Updated_POI_schedule_north_west", 
                        status='Active', 
                        wait_for_provisioning=True,
                        recurrence=recurrence)

fetched_schedule = Schedule.get(ws, fetched_schedule.id)

print("Updated schedule:", fetched_schedule.id, 
      "\nNew name:", fetched_schedule.name,
      "\nNew frequency:", fetched_schedule.recurrence.frequency,
      "\nNew status:", fetched_schedule.status)

Provisioning status: Completed
Updated schedule: 76466852-b1bf-4668-bd5e-e522ed03c437 
New name: POI_schedule_north_west 
New frequency: Minute 
New status: Active


In [116]:
#disable all
ss = Schedule.list(ws)
for s in ss:
    s.disable()
print(ss)

[Pipeline(Name: POI_schedule,
Id: f0b7f2ac-18cc-43e2-881f-a2fe7c4b6b06,
Status: Disabled,
Pipeline Id: 286065c9-2a98-4ae5-8756-0118ee3c0d79,
Recurrence Details: Runs every 45 Minutes)]
